# Pohlig Hellman Attack on Elliptic Curves

## Preliminaries

An elliptic curve E over a field K of characteristic different than 2 or 3 is given by:

$$
E(K) = \{ (x,y) \in K x K : y^2 = x^3+ax+b \}
$$

Elliptic curves of this type over fields $F_p$, where $p$ is prime, are modeled in the module 'elliptic.py'.

Here are some examples of the operations that you can perform on elliptic curves:

In [1]:
from elliptic import *

ec = EllipticCurve(4, 20, 29)
print(ec)

y^2 = x^3 + 4x + 20 (mod 29)


In [2]:
# compute the determinant

ec.determinant()

4

In [3]:
# Check if a point belongs to the curve

ec.is_valid_point((5, 22))

True

In [4]:
# add/subtract/double and negate

result = ec.add((5, 22), (16, 27))
assert result == (13, 6)

result = ec.subtract((13, 6), (5, 22))
assert result == (16, 27)

result = ec.double((5, 22))
assert result == (14, 6)

assert ec.negate((5, 22)) == (5, 7)

The point at infinity (the group identity) is represented by None

In [5]:
result = ec.add((1, 5), (1, 24)) 
print(result)

None


In [6]:
ec.add(None, (5, 22))

(5, 22)

You can multiply a point with an integer to represent adding a point to itself n times

In [7]:
p = (1, 5)
assert ec.is_valid_point(p)
assert ec.multiply(p, 2) == (4, 19)
assert ec.multiply(p, 21) == (0, 7)

## The Discrete Logarithm in Elliptic Curves

Given a curve $E(F_p)$, pick a point $P \in E(F_p)$ with known order $n$. Let $Q \in <P>$. The discrete logarithm problem is to find $l: Q = lP$ 

### The Pohlig-Hellman attack

To effect the PH attack, we need the following prerequisites:

1. The ability to factor the order $n$ of $P$ into primes with their multiplicity

2. The ability to mount a successful attack for every prime that divides $n$.

Neither of these prerequisites can be guaranteed if $n$ is very large, but for the small problems in the book we can use naive algorithms to achieve them.


Factorization methods can be found in the module factors.py

Here is an example:


In [8]:
from factors import *

naive_factorization_with_multiplicity(8661340972)

[(2, 2), (2165335243, 1)]

The naive version of Pollard's rho attack described in the beginning of section 4.1.2 is implemented in elliptic_attacks.py

In [10]:
from elliptic_attacks import *

ec = EllipticCurve(1, 44, 229)
order = 239
p = (5, 116)
q = (155, 166)
naive_prime_attack(ec, p, order, q)

176

In [11]:
ec.multiply(p, 176)

(155, 166)

In [13]:
# Here is the code for the naive prime attack

def naive_prime_attack(ec: EllipticCurve, p: Point, prime_order: int, q: Point,
                       iterations: int = 10000) -> int:
    assert is_probably_prime(prime_order), "The order must be a prime number"
    if p == q:
        return 1

    def find_dupes() -> tuple[int, int, int, int] | None:
        lookup: Dict[Point, tuple[int, int]] = {}
        for _ in range(iterations):
            c = randint(1, prime_order - 1)
            d = randint(1, prime_order - 1)
            result = ec.add(ec.multiply(p, c), ec.multiply(q, d))
            if result in lookup:
                e, f = lookup[result]
                if e != c or f != d:
                    return c, d, e, f
            lookup[result] = (c, d)
        return None

    result = find_dupes()
    if not result:
        raise ValueError("No solution found")
    c, d, e, f = result
    a = (c-e) % prime_order
    b = (f-d) % prime_order
    return a * mod_inverse(b, prime_order) % prime_order


Finally, a solver for a system of mod equations using the Chinese Remainder Theorem, can be found in integers.py:


In [12]:
from integers import solve_chinese_remainder

remainders = [(6, 11), (13, 16), (9, 21), (19, 25)]
solve_chinese_remainder(remainders)

89469

The Pohlig Hellman attack is implemented as a class in elliptic_attacks.py. 

Here is how to use it:


In [14]:
ec = EllipticCurve(1001, 75, 7919)
order = 7889
p = (4023, 6036)
q = (4135, 3169)
ph = PohlighHellman(ec, p, order, q)
ph.solve()

4334